## 模型性能

In [3]:
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import *
import numpy as np
sys.path.append('../data/')
sys.path.append("../")
from load_cifar import load_cifar
from load_olivetti import load_olivetti
from resnet import resnet
from preact_resnet import PreActResNet18

In [4]:
os.listdir('ckpts/')

['olivetti_sgn_preact_8.0_model.ckpt',
 'olivetti_sgn_preact_32.0_model.ckpt',
 'cifar_sgn_preact_resnet_8.0_model.ckpt',
 'cifar_sgn_res5_8.0_model.ckpt',
 'cifar_sgn_res5_16.0_model.ckpt',
 'olivetti_sgn_resnet_768.0_model.ckpt',
 'cifar_sgn_res5_15.0_model.ckpt',
 'olivetti_sgn_resnet_2048.0_model.ckpt',
 'cifar_sgn_res5_32.0_model.ckpt',
 'cifar_sgn_preact_resnet_16.0_model.ckpt',
 'olivetti_sgn_resnet_1024.0_model.ckpt',
 'olivetti_sgn_preact_16.0_model.ckpt',
 'cifar_sgn_preact_resnet_32.0_model.ckpt',
 'olivetti_sgn_resnet_32.0_model.ckpt',
 'olivetti_sgn_resnet_8.0_model.ckpt',
 'olivetti_sgn_resnet_512.0_model.ckpt',
 'olivetti_sgn_resnet_1536.0_model.ckpt',
 'olivetti_sgn_resnet_16.0_model.ckpt']

#### CIFAR10/ResNet34

In [5]:
# 加载数据
def reshape_data(X_train, y_train, X_test):
    # reshape train and subtract mean
    pixel_mean = np.mean(X_train, axis=0)
    X_train -= pixel_mean
    X_test -= pixel_mean
    return X_train, y_train, X_test

def load_data(name='cifar'):
    if name == 'cifar':
        X_train, y_train, X_test, y_test = load_cifar(10)
        X_train = np.dstack((X_train[:, :1024], X_train[:, 1024:2048], X_train[:, 2048:]))
        X_train = X_train.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
        X_test = np.dstack((X_test[:, :1024], X_test[:, 1024:2048], X_test[:, 2048:]))
        X_test = X_test.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
    else:
        X_train, y_train, X_test, y_test = load_olivetti()
    return X_train, y_train, X_test, y_test

def rbg_to_grayscale(images):
    return np.dot(images[..., :3], [0.299, 0.587, 0.114])

In [6]:
def data_to_loader(x, y, size=512):
    x_data = torch.from_numpy(x)
    y_data = torch.from_numpy(y).long()
    loader = DataLoader(dataset=TensorDataset(x_data, y_data), 
                          batch_size=size, 
                          shuffle=False, 
                          num_workers=2)
    return loader

In [7]:
# 加载CIFAR10
X_train, y_train, X_test, y_test = load_data(name='cifar')
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
test_loader = data_to_loader(X_test, y_test)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
df_metric = pd.DataFrame(columns=('网络','数据集', 'lambda', '准确率', 'micro-F1', 'macro-F1'))

In [9]:
def metric_model(net, modelpath, data_loader, lam, netname='ResNet34', dataname='CIFAR10'):
    net.load_state_dict(torch.load(modelpath))
    res = []
    y_data = []
    network.eval()
    for i, data in enumerate(data_loader):
        x, y = data
        x_device = x.to(device)
        pred = network(x_device)
        res.append(pred.max(1)[1])
        y_data.append(y.to(device))
    res = torch.cat(res)
    y_data = torch.cat(y_data)
    print(len(res))
    # 准确率
    acc = accuracy_score(res.cpu().numpy(), y_data.cpu().numpy())
    # micro-f1
    micro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='micro')
    # macro-f1
    macro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='macro')
    
    global df_metric
    # TODO:修改网络和数据集名字
    df_metric = df_metric.append({
        '网络': netname, 
        '数据集': dataname,
        'lambda': lam,
        '准确率': acc,
        'micro-F1': micro_f1,
        'macro-F1': macro_f1
    }, ignore_index=True)
#     df_metric.to_latex(sys.stderr)

In [11]:
# CIFAR10/ResNet34/4.0
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_res5_8.0_model.ckpt'
metric_model(network, path, test_loader, 8.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
\bottomrule
\end{tabular}


In [13]:
# CIFAR10/ResNet34/8.0
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_res5_16.0_model.ckpt'
metric_model(network, path, test_loader, 16.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
1 &  ResNet34 &  CIFAR10 &    16.0 &  0.9223 &    0.9223 &  0.922307 \\
\bottomrule
\end{tabular}


In [14]:
# CIFAR10/ResNet34/16.0
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_res5_32.0_model.ckpt'
metric_model(network, path, test_loader, 32.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
1 &  ResNet34 &  CIFAR10 &    16.0 &  0.9223 &    0.9223 &  0.922307 \\
2 &  ResNet34 &  CIFAR10 &    32.0 &  0.9166 &    0.9166 &  0.916605 \\
\bottomrule
\end{tabular}


In [18]:
# CIFAR10/ResNet34/16.0
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_preact_resnet_8.0_model.ckpt'
metric_model(network, path, test_loader, 8.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
1 &  ResNet34 &  CIFAR10 &    16.0 &  0.9223 &    0.9223 &  0.922307 \\
2 &  ResNet34 &  CIFAR10 &    32.0 &  0.9166 &    0.9166 &  0.916605 \\
3 &  ResNet34 &  CIFAR10 &     8.0 &  0.9413 &    0.9413 &  0.941216 \\
\bottomrule
\end{tabular}


In [19]:
# CIFAR10/ResNet34/16.0
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_preact_resnet_16.0_model.ckpt'
metric_model(network, path, test_loader, 16.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
1 &  ResNet34 &  CIFAR10 &    16.0 &  0.9223 &    0.9223 &  0.922307 \\
2 &  ResNet34 &  CIFAR10 &    32.0 &  0.9166 &    0.9166 &  0.916605 \\
3 &  ResNet34 &  CIFAR10 &     8.0 &  0.9413 &    0.9413 &  0.941216 \\
4 &  ResNet34 &  CIFAR10 &    16.0 &  0.9464 &    0.9464 &  0.946308 \\
\bottomrule
\end{tabular}


In [20]:
# CIFAR10/ResNet34/16.0
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_sgn_preact_resnet_32.0_model.ckpt'
metric_model(network, path, test_loader, 32.0 ,netname='ResNet34', dataname='CIFAR10')
df_metric.to_latex(sys.stderr)

10000


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &      数据集 &  lambda &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &     8.0 &  0.9259 &    0.9259 &  0.925786 \\
1 &  ResNet34 &  CIFAR10 &    16.0 &  0.9223 &    0.9223 &  0.922307 \\
2 &  ResNet34 &  CIFAR10 &    32.0 &  0.9166 &    0.9166 &  0.916605 \\
3 &  ResNet34 &  CIFAR10 &     8.0 &  0.9413 &    0.9413 &  0.941216 \\
4 &  ResNet34 &  CIFAR10 &    16.0 &  0.9464 &    0.9464 &  0.946308 \\
5 &  ResNet34 &  CIFAR10 &    32.0 &  0.9461 &    0.9461 &  0.946063 \\
\bottomrule
\end{tabular}


## Olivetti

In [22]:
# 加载Olivetti
X_train, y_train, X_test, y_test = load_data(name='olivetti')
# mal_x_2, mal_y_2, num_targets = mal_data_synthesis(X_train, num_targets=2)
# mal_x_4, mal_y_4, num_targets = mal_data_synthesis(X_train, num_targets=4)
# mal_x_8, mal_y_8, num_targets = mal_data_synthesis(X_train, num_targets=8)
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
test_loader = data_to_loader(X_test, y_test)
# mal_2_loader = data_to_loader(mal_x_2, mal_y_2)
# mal_4_loader = data_to_loader(mal_x_4, mal_y_4)
# mal_8_loader = data_to_loader(mal_x_8, mal_y_8)

In [23]:
df_metric = pd.DataFrame(columns=('网络','数据集', '准确率', 'micro-F1', 'macro-F1'))

In [24]:
# 2
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_resnet_8.0_model.ckpt'
metric_model(network, path, test_loader, 8.0 ,netname='ResNet34', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &       数据集 &  准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &  ResNet34 &  Olivetti &  0.9 &       0.9 &  0.896825 &     8.0 \\
\bottomrule
\end{tabular}


In [25]:
# 2
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_resnet_16.0_model.ckpt'
metric_model(network, path, test_loader, 16.0 ,netname='ResNet34', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &  ResNet34 &  Olivetti &  0.900000 &  0.900000 &  0.896825 &     8.0 \\
1 &  ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.913810 &    16.0 \\
\bottomrule
\end{tabular}


In [26]:
# 2
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_resnet_32.0_model.ckpt'
metric_model(network, path, test_loader, 32.0 ,netname='ResNet34', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &        网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &  ResNet34 &  Olivetti &  0.900000 &  0.900000 &  0.896825 &     8.0 \\
1 &  ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.913810 &    16.0 \\
2 &  ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.911310 &    32.0 \\
\bottomrule
\end{tabular}


In [27]:
# 2
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_preact_8.0_model.ckpt'
metric_model(network, path, test_loader, 8.0 ,netname='PreActResNet18', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &              网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &        ResNet34 &  Olivetti &  0.900000 &  0.900000 &  0.896825 &     8.0 \\
1 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.913810 &    16.0 \\
2 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.911310 &    32.0 \\
3 &  PreActResNet18 &  Olivetti &  0.891667 &  0.891667 &  0.886310 &     8.0 \\
\bottomrule
\end{tabular}


In [28]:
# 2
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_preact_16.0_model.ckpt'
metric_model(network, path, test_loader, 16.0 ,netname='PreActResNet18', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &              网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &        ResNet34 &  Olivetti &  0.900000 &  0.900000 &  0.896825 &     8.0 \\
1 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.913810 &    16.0 \\
2 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.911310 &    32.0 \\
3 &  PreActResNet18 &  Olivetti &  0.891667 &  0.891667 &  0.886310 &     8.0 \\
4 &  PreActResNet18 &  Olivetti &  0.908333 &  0.908333 &  0.903452 &    16.0 \\
\bottomrule
\end{tabular}


In [29]:
# 2
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_sgn_preact_32.0_model.ckpt'
metric_model(network, path, test_loader, 32.0 ,netname='PreActResNet18', dataname='Olivetti')
df_metric.to_latex(sys.stderr)

120


\begin{tabular}{lllrrrr}
\toprule
{} &              网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 &  lambda \\
\midrule
0 &        ResNet34 &  Olivetti &  0.900000 &  0.900000 &  0.896825 &     8.0 \\
1 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.913810 &    16.0 \\
2 &        ResNet34 &  Olivetti &  0.916667 &  0.916667 &  0.911310 &    32.0 \\
3 &  PreActResNet18 &  Olivetti &  0.891667 &  0.891667 &  0.886310 &     8.0 \\
4 &  PreActResNet18 &  Olivetti &  0.908333 &  0.908333 &  0.903452 &    16.0 \\
5 &  PreActResNet18 &  Olivetti &  0.891667 &  0.891667 &  0.888988 &    32.0 \\
\bottomrule
\end{tabular}
